# DAQC VQE Challenge – Using DAQC in Qiskit

Here, we provide an example of how to create an analog block with Qiskit. The most important part in the Qiskit documentation for this is the [HamiltonianGate](https://qiskit.org/documentation/stubs/qiskit.extensions.HamiltonianGate.html) class.

In [1]:
# We will need some functionality 
from typing import List 

# and from math related libraries
import numpy as np
import qutip as qt

# and from qiskit
from qiskit.extensions import HamiltonianGate
from qiskit import QuantumCircuit, QuantumRegister, Aer, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Operator

## Creating a nearest neighbour Hamiltonian for a given connectivity

We now need a function that creates our Hamiltonian for a given connectivity

- A Hamiltonian for a Ising model in 3-dimentions

In [2]:
# if Julia is not installed run this line
# !pip install juliacall

In [3]:
from juliacall import Main as jl;

In [4]:
jl.pwd() # check the current directory for Julia

'd:\\Cesar\\GitHub\\Womanium_hack\\iqm-academy-womanium-hackathon-DAQC-VQE'

In [5]:
#jl.cd("/iqm-academy-womanium-hackathon-DAQC-VQE/")   # Change the current julia directory if necessary 
jl.include("Ising_model.jl")  # Load the julia script with the connectivity generator function

create_zz_hamiltonian (generic function with 3 methods)

In [6]:
# Creation of the Phyton call of the Julia version

# Recives a tuple of integers of length 3 and returns a list of lists with the connectivities of the Issing Hamiltinian
def connectivity_Ising(dim_lengths):    
    connectivity_julia = jl.connectivity(dim_lengths)
    connectivity = []
    for i in range(len(connectivity_julia)):
        connectivity.append(list(connectivity_julia[i]))
    return connectivity

# Recives a tuple of integers of length 3 and returns a Hamiltonian matrix, a number of qubits, a uniform list of
# interaction coefficients, and a list of connectivities. 
def Ising_matrix(dim_lengths):
    matrix, no_qubits, h_coeffs, connect_jl = jl.create_zz_hamiltonian(dim_lengths)
    h_coeffs = list(h_coeffs)
    matrix = (matrix.__array__()).tolist()
    connect = []
    for i in range(len(connect_jl)):
        connect.append(list(connect_jl[i]))

    return matrix, no_qubits, h_coeffs, connect

In [7]:
connectivity_Ising((2,2,1))

[[0, 1], [0, 2], [1, 3], [2, 3]]

In [8]:
# ***** The biggest tuple that we can handle is for a lattice of (2,2,3) dimentions!!!   <<<<<<<----- IMPORTANT
issing_ham, issing_no_qubits, issing_h_coeffs, issing_connectivities = Ising_matrix((4,1,1)) 

In [9]:
issing_no_qubits

4

In [10]:
print(issing_ham)

[[(3+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, (-3+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-3+0j), 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j], [0j, 0j, 0j

In [11]:
issing_connectivities

[[0, 1], [1, 2], [2, 3]]

In [12]:
issing_h_coeffs

[1.0, 1.0, 1.0]

- A Issing Hamiltonian given a connectivity

In [76]:
# Recives a list of connectivities, returns the hamiltonian, number_of_qubits, h_coeffs and the connectivity provided. 
def given_Issing_ham(connectivity):
    a0 = []
    for i in connectivity:
        a0.append(jl.PythonCall.pyconvert(jl.Vector,i))
        j0 = jl.PythonCall.pyconvert(jl.Vector,a0)
    
    jl.convert(jl.Vector,a0)
    hamiltonian, num_qubits, h_coeffs, connectivities = jl.create_zz_hamiltonian(j0)
    hamiltonian = (hamiltonian.__array__()).tolist()
    h_coeffs = (h_coeffs.__array__()).tolist()

    return hamiltonian, num_qubits, h_coeffs, connectivity

In [77]:
issing_ham_con, issing_num_qubits_con, issing_h_coeffs_con, issing_con_con = given_Issing_ham([[0,1],[1,2],[2,3]])

In [78]:
print(list(issing_ham_con))

[[(3+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, (-3+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-3+0j), 0j, 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (1+0j), 0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, 0j, (-1+0j), 0j, 0j], [0j, 0j, 0j

In [79]:
issing_num_qubits_con

4

In [80]:
issing_h_coeffs_con

[1.0, 1.0, 1.0, 1.0]

In [81]:
issing_con_con

[[0, 1], [1, 2], [2, 3]]

## Use multiple parameters for the Hamiltonian

As the standard implementation of HamiltonianGate in qiskit does not support parameters for the Hamiltonian, we need our own Gate implementation.

In [4]:
from numbers import Number
import numpy

import qutip as qt

from qiskit.circuit import Gate, QuantumCircuit, QuantumRegister, ParameterExpression
from qiskit.quantum_info.operators.predicates import matrix_equal
from qiskit.quantum_info.operators.predicates import is_hermitian_matrix
from qiskit.extensions.exceptions import ExtensionError
from qiskit.circuit.exceptions import CircuitError

from qiskit.circuit import Parameter, QuantumCircuit, QuantumRegister 


from qiskit.extensions.unitary import UnitaryGate

# This code is based on https://github.com/Qiskit/qiskit-terra/blob/main/qiskit/extensions/hamiltonian_gate.py licenced under Apache licence.
class CustomHamiltonianGate(Gate):
    def __init__(self, time, no_qubits, connectivity, h_coeff0,h_coeff1,h_coeff2,h_coeff3=1.0,h_coeff4=1.0,h_coeff5=1.0, label=None):
        if isinstance(time, Number) and time != numpy.real(time):
            raise ExtensionError("Evolution time is not real.")
       
        self.no_qubits = no_qubits
        self.connectivity = connectivity
        # Store instruction params
        super().__init__("custom_hamiltonian", no_qubits, [time, no_qubits, connectivity, h_coeff0,h_coeff1,h_coeff2,h_coeff3,h_coeff4,h_coeff5], label=label)

    def __array__(self, dtype=None):
        """Return matrix for the unitary."""
        # pylint: disable=unused-argument
        import scipy.linalg

        try:
            return scipy.linalg.expm(-1j * self.get_ham() * float(self.params[0]))
        except TypeError as ex:
            raise TypeError(
                "Unable to generate Unitary matrix for "
                "unbound t parameter {}".format(self.params[1])
            ) from ex

    def _define(self):
        """Calculate a subcircuit that implements this unitary."""
        q = QuantumRegister(self.no_qubits, "q")
        qc = QuantumCircuit(q, name=self.name)
        qc._append(UnitaryGate(self.to_matrix()), q[:], [])
        self.definition = qc

    def validate_parameter(self, parameter):
        return parameter

    def get_ham(self):
        dim = 2 ** self.no_qubits
        no_connections = len(self.connectivity)
        zz_hamiltonian = np.zeros([dim, dim], dtype=np.complex128)

        for c in range(no_connections):
            ops_to_tensor = [qt.identity(2)] * self.no_qubits
            ops_to_tensor[self.connectivity[c][0]] = qt.sigmaz()
            ops_to_tensor[self.connectivity[c][1]] = qt.sigmaz()
            zz_hamiltonian += float(self.params[3+c]) * np.array(qt.tensor(ops_to_tensor)) #+= 

        return zz_hamiltonian

def custom_hamiltonian(self, time, connectivity,qubits,h_coeff0,h_coeff1,h_coeff2,h_coeff3=1.0,h_coeff4=1.0,h_coeff5=1.0, label=None):
    """Apply hamiltonian evolution to qubits."""
    if not isinstance(qubits, list):
        qubits = [qubits]

    return self.append(CustomHamiltonianGate(time=time, no_qubits = len(qubits), connectivity=connectivity, h_coeff0=h_coeff0,h_coeff1=h_coeff1,h_coeff2 = h_coeff2,h_coeff3 = h_coeff3,h_coeff4 = h_coeff4,h_coeff5 = h_coeff5, label=label), qubits, [])


QuantumCircuit.custom_hamiltonian = custom_hamiltonian

In [7]:
import numpy as np
import pylab

from qiskit import Aer
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit import  ParameterVector
from qiskit.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import SPSA
from qiskit.opflow import I, X, Z
import os
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.fake_provider import FakeVigo

def vqe_with_daqc(no_qubits = 4, no_iters = 1, connections = [[0,1],[1,2],[2,3]]):
    if no_qubits == 4:
        H2_op = (-1.052373245772859 * I ^ I ^ I ^I ) + \
        (0.39793742484318045 * I ^ Z ^ I ^I ) + \
        (-0.39793742484318045 * Z ^ I ^ I ^I ) + \
        (-0.01128010425623538 * Z ^ Z ^ I ^I ) + \
        (0.18093119978423156 * X ^ X ^ I ^I )
    elif no_qubits == 2:
        H2_op = (-1.052373245772859 * I ^ I) + \
        (0.39793742484318045 * I ^ Z) + \
        (-0.39793742484318045 * Z ^ I) + \
        (-0.01128010425623538 * Z ^ Z) + \
        (0.18093119978423156 * X ^ X)
    else: 
        print("Error: Number of qubits must be 2 or 4.")

    seed = 170
    iterations = 125
    algorithm_globals.random_seed = seed
    backend = Aer.get_backend('aer_simulator')
    qi = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed) 
    counts = []
    values = []
    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    no_con = len(connections)
    p = ParameterVector('p', no_con + 3 * no_qubits + no_iters * (3 * no_qubits + 1))

    qr = QuantumRegister(no_qubits)
    circ = QuantumCircuit(qr)

    for i in range(no_qubits):
        circ.u(p[3*i],p[3*i+1],p[3*i+2],i)
    for j in range(no_iters):
        n = 3 * no_qubits
        if no_con == 1 and no_qubits == 2:
            circ.custom_hamiltonian(connectivity=connections, h_coeff0=p[n], h_coeff1=1.0, h_coeff2=1.0, time=p[n+1+13*j], qubits=[qr[0], qr[1]], label='analog block')
        elif no_con == 3 and no_qubits == 4:
            circ.custom_hamiltonian(connectivity=connections, h_coeff0=p[n], h_coeff1=p[n+1], h_coeff2=p[n+2], time=p[n+3+13*j], qubits=[qr[0], qr[1], qr[2], qr[3]], label='analog block')
        elif no_con == 4 and no_qubits == 4:
            circ.custom_hamiltonian(connectivity=connections, h_coeff0=p[n], h_coeff1=p[n+1], h_coeff2=p[n+2], h_coeff3=p[n+3], time=p[n+4+13*j], qubits=[qr[0], qr[1], qr[2], qr[3]], label='analog block')
        elif no_con == 5 and no_qubits == 4:
            circ.custom_hamiltonian(connectivity=connections, h_coeff0=p[n], h_coeff1=p[n+1], h_coeff2=p[n+2], h_coeff3=p[n+3], h_coeff4=p[n+4], time=p[n+5+13*j], qubits=[qr[0], qr[1], qr[2], qr[3]], label='analog block')
        elif no_con == 6 and no_qubits == 4:
            circ.custom_hamiltonian(connectivity=connections, h_coeff0=p[n], h_coeff1=p[n+1], h_coeff2=p[n+2], h_coeff3=p[n+3], h_coeff4=p[n+4], h_coeff5=p[n+5], time=p[n+6+13*j], qubits=[qr[0], qr[1], qr[2], qr[3]], label='analog block')
        else:
            print("Error: Number of connections must be 1 for 2 qubits or 3, 4, 5 or 6 for 4 qubits.")
        for i in range(no_qubits):
            n = no_con + 3 * no_qubits + (3 * no_qubits + 1) * j + 3 * i + 1
            circ.u(p[n],p[n+1],p[n+2],i)
    ansatz = circ

    spsa = SPSA(maxiter=iterations)
    vqe = VQE(ansatz, optimizer=spsa, callback=store_intermediate_result, quantum_instance=qi)
    result = vqe.compute_minimum_eigenvalue(operator=H2_op)
    res = result.eigenvalue.real
    return res
    print(ansatz)

In [8]:
ref_value = -1.85728
print(f'Reference value: {ref_value:.5f}')
for k in range(1,3):
    res = vqe_with_daqc(4,k)
    print(f"k={k}")
    print(f'VQE on Aer qasm simulator (no noise): {res:.5f}')
    print(f'Delta from reference energy value is {(res - ref_value):.5f}')
    

Reference value: -1.85728
k=1
VQE on Aer qasm simulator (no noise): -1.83965
Delta from reference energy value is 0.01763
k=2
VQE on Aer qasm simulator (no noise): -1.83859
Delta from reference energy value is 0.01869
